In [ ]:
from grid.refine import *
import jittens

In [ ]:
runs.exists()

In [ ]:
container_command('boardlaw-1')

In [ ]:
jittens.manage.machine_run('ls /code')

In [ ]:
m = jittens.machines.machines()['boardlaw-1']

In [ ]:
jittens.jobs.submit('python -c "from grid.refine import *" >logs.txt 2>&1', dir='.', resources={})

In [ ]:
jittens.manage.refresh()

In [ ]:
a.get(project=PROJECT, zone=ZONE, machineType=machine).execute()

In [ ]:
i.list??